<a href="https://colab.research.google.com/github/megan0422/DABPFinalProject/blob/main/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
!pip install gurobipy
import gurobipy as gp
from gurobipy import *
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from random import randrange
import seaborn as sns
import warnings
import math
warnings.filterwarnings("ignore")

In [ ]:
# NBI data
PA22 = pd.read_csv("PA22.csv")
# Extract data on allegheny county from the whole dataset
PA22_allegheny  = PA22[PA22["COUNTY_CODE_003"] == 3]
PA22_allegheny  = PA22_allegheny[PA22_allegheny["MAINTENANCE_021"] == 2]
PA22_allegheny

In [ ]:
PA22_allegheny = PA22_allegheny[["STRUCTURE_NUMBER_008","ADT_029","STRUCTURE_LEN_MT_049","TOTAL_IMP_COST_096","HIGHWAY_SYSTEM_104","PERCENT_ADT_TRUCK_109"]]
PA22_allegheny.dropna()
PA22_allegheny_cost_nonzero = PA22_allegheny[PA22_allegheny["TOTAL_IMP_COST_096"] != 0]
PA22_allegheny_cost_nonzero.describe()

In [ ]:

i = len(PA22_allegheny_cost_nonzero)
bridge = range(i)

# Step 0. Set up the model
model = Model()

# Step 1. Set up the decision variables
x = model.addVars(bridge, vtype=GRB.BINARY)

addtr = (PA22_allegheny_cost_nonzero["PERCENT_ADT_TRUCK_109"].to_numpy())/100
adt = PA22_allegheny_cost_nonzero["ADT_029"].to_numpy()
bl = PA22_allegheny_cost_nonzero["STRUCTURE_LEN_MT_049"].to_numpy()
hwy = PA22_allegheny_cost_nonzero["HIGHWAY_SYSTEM_104"].to_numpy()

#Step 2. Set up the objective function
T = addtr*adt/24*32*1 + addtr*adt/24*16.2*1.67
V = adt*bl*hwy*50*(3.02+0.94) + adt*bl*(1-hwy)*25*(3.02+0.45)

model.setObjective (sum((T[i]+V[i])*x[i] for i in bridge))
model.modelSense = GRB.MAXIMIZE

#Step 3. Set up the constraints
#budget constraint
C = PA22_allegheny_cost_nonzero["TOTAL_IMP_COST_096"].to_numpy()*1000
B = 27700000

model.addConstr(sum(C[i]*x[i] for i in bridge) <= B) 
model.optimize()

In [ ]:
decision_variable = []
for v in model.getVars():
    decision_variable.append(v.x)
print("Total cost is", np.nansum(np.array(decision_variable)*C))

In [ ]:
structure_number_with_zero = np.array(decision_variable)*PA22_allegheny_cost_nonzero["STRUCTURE_NUMBER_008"].to_numpy().astype(int)
structure_number = structure_number_with_zero[structure_number_with_zero != 0]
bridge_number = len(structure_number)
print(bridge_number, "bridges should be repair/replacement")
print("The STRUCTURE NUMBER of the bridges to repair/replace are",structure_number)